In [1]:
from acfunctions import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.stats as astat
from solartwins import *
import pickle
from IPython.display import Image, display, HTML
import starspot as ss
import starspot.rotation_tools as rt
from starspot import sigma_clipping
import lightkurve as lk
from lightkurve import search_targetpixelfile, search_lightcurvefile
import math
import exoplanet
import eleanor
from astropy.coordinates import SkyCoord
from scipy import interpolate
from tess_kep_funct import *
from tqdm import tqdm
from scipy import misc
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
import warnings
warnings.filterwarnings("ignore")

/Users/ilanadoran/opt/anaconda3/lib/python3.7/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=False)

In [3]:
output_notebook()

Loading BokehJS ...

In [5]:
#dataset with tess vs kepler IDs
tess_kep = pd.read_csv('data_summary.csv')
tess_short = tess_kep[tess_kep.tess_ffi == True]
cat = pd.read_csv('tics_kics_mcq_rot.txt', names=['tic_id', 'kic_id', 'prot'], delim_whitespace=True)

In [6]:
#importing McQ periods
headers = 'TIC ID', 'KIC ID', 'McQ Periods';  

with open("tics_kics_mcq_rot.txt") as file:
    rot_table = [line.split() for line in file.read().splitlines()]

widths = [max(len(value) for value in col)
for col in zip(*(rot_table + [headers]))]

formatting = '{:{widths[0]}}  {:{widths[1]}}  {:{widths[2]}}'
formatting.format(*headers, widths=widths)
for tics_kics_mcq in rot_table:
    formatting.format(*tics_kics_mcq, widths=widths)

In [7]:
# converting txt file data to arrays
tic_ids = []
mcq_periods = []
kic_ids = []

for index, row in enumerate(rot_table):
    tic_ids.append(int(rot_table[index][0]))
    kic_ids.append(int(rot_table[index][1]))
    mcq_periods.append(float(rot_table[index][2]))

In [12]:
# selecting shortest periods only (<13 days, half a tess cycle)
short_periods = []
tics_short_periods = []
kics_short_periods = []

for index, period in enumerate(mcq_periods):
    if period < 13: 
        short_periods.append(period)
        tics_short_periods.append(tic_ids[index])
        kics_short_periods.append(kic_ids[index])
        
#limiting to the first 60
short_periods = short_periods[0:]
tics_short_periods = tics_short_periods[0:61]
kics_short_periods = kics_short_periods[0:61]

ls_kep = []
pdm_kep = []
exo_kep = []

for index, value in (enumerate(kics_short_periods)):
    period_kep = kep_ac(value)
    ls_kep.append(period_kep[0])
    pdm_kep.append(period_kep[1])
    exo_kep.append(period_kep[2])

smallest_diff = []
method = []

for each, mcq in enumerate(short_periods): 
    diff_ls = np.abs(ls_kep[each] - mcq)
    diff_pdm = np.abs(pdm_kep[each] - mcq)
    diff_exo = np.abs(exo_kep[each] - mcq)
    
    index = np.argmin([diff_ls, diff_pdm, diff_exo])
    
    if index == 0: 
        smallest_diff.append(ls_kep[each])
        method.append('lomb-scargle')  
    
    if index == 1: 
        smallest_diff.append(pdm_kep[each])
        method.append('PDM')
            
    if index == 2 :
        smallest_diff.append(exo_kep[each])
        method.append('exoplanet acf')

with open('short_periods.data', 'wb') as filehandle:
    pickle.dump(short_periods, filehandle)

with open('tic_ids.data', 'wb') as filehandle:
    pickle.dump(tic_ids, filehandle)

In [8]:
with open('kic_ids.data', 'wb') as filehandle:
    pickle.dump(kic_ids, filehandle)

with open('smallest_diff.data', 'wb') as filehandle:
    pickle.dump(smallest_diff, filehandle)

with open('method.data', 'wb') as filehandle:
    pickle.dump(method, filehandle)

with open('ls_kep.data', 'wb') as filehandle:
    pickle.dump(ls_kep, filehandle)

with open('pdm_kep.data', 'wb') as filehandle:
    pickle.dump(pdm_kep, filehandle)

with open('exo_kep.data', 'wb') as filehandle:
    pickle.dump(exo_kep, filehandle)

with open('smallest_diff.data', 'rb') as filehandle:
    smallest_diff = pickle.load(filehandle) 

with open('method.data', 'rb') as filehandle:
    method = pickle.load(filehandle) 

with open('ls_kep.data', 'rb') as filehandle:
    ls_kep = pickle.load(filehandle) 

with open('pdm_kep.data', 'rb') as filehandle:
    pdm_kep = pickle.load(filehandle) 

with open('exo_kep.data', 'rb') as filehandle:
    exo_kep = pickle.load(filehandle) 

In [9]:
print(ls_kep[1], exo_kep[1], pdm_kep[1], short_periods[1])

NameError: name 'ls_kep' is not defined

In [ ]:
color_coded_periods = []
mcq_color_ordered = []

for each, mcq in enumerate(short_periods): 
    if method[each] == 'PDM':
        color_coded_periods.append(smallest_diff[each])
        mcq_color_ordered.append(mcq)
        
for each, mcq in enumerate(short_periods): 
    if method[each] == 'lomb-scargle':
        color_coded_periods.append(smallest_diff[each])
        mcq_color_ordered.append(mcq)
        
for each, mcq in enumerate(short_periods): 
    if method[each] == 'exoplanet acf':
        color_coded_periods.append(smallest_diff[each])
        mcq_color_ordered.append(mcq)

In [ ]:
print(len(mcq_color_ordered[24:45]))

In [ ]:
x =  mcq_color_ordered
y = color_coded_periods

source = ColumnDataSource(data=dict(
    x =  mcq_color_ordered,
    y = color_coded_periods,
    pdm = color_coded_periods[0:24], 
    ls = color_coded_periods[24:45],
    exo = color_coded_periods[45:],
    kep = (cat['kic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

kep_colored_periods_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title=" Kepler Periods, Best Method",
   x_axis_label='McQuillan Period', y_axis_label='Kepler Period', plot_width=900)

kep_colored_periods_plot.line(x, x, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=x", line_width = 2)
kep_colored_periods_plot.circle('x', 'pdm', source = source, color="mediumvioletred", line_color=None)
kep_colored_periods_plot.circle('x', 'ls', source = source, color="black", line_color=None)
kep_colored_periods_plot.circle('x', 'exo', source = source, color="orange", line_color=None)

kep_colored_periods_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)")] 

show(kep_colored_periods_plot)

In [ ]:
x =  short_periods
y = smallest_diff

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = smallest_diff, 
    kep = (cat['kic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

kep_periods_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title=" Kepler Periods, Best Method",
   x_axis_label='McQuillan Period', y_axis_label='Kepler Period', plot_width=900)

kep_periods_plot.line(x, x, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=x", line_width = 2)
kep_periods_plot.circle('x', 'y', source = source, color="mediumvioletred", line_color=None)
kep_periods_plot.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
kep_periods_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'pink', line_dash="dotted")

kep_periods_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("KIC ID", "@tess")] 

show(kep_periods_plot)

In [ ]:
x =  short_periods
y = exo_kep

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = exo_kep, 
    kep = (cat['kic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

kep_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title=" Kepler Periods, Best Method",
   x_axis_label='McQuillan Period', y_axis_label='Kepler Period', plot_width=900)

kep_plot.line(x, x, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=x", line_width = 2)
kep_plot.circle('x', 'y', source = source, color="mediumvioletred", line_color=None)
kep_plot.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
kep_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'pink', line_dash="dotted")

kep_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("KIC ID", "@tess")] 

show(kep_plot)

In [13]:
short_periods[11]

10.68

In [10]:
ls_tess = []
pdm_tess = []
exo_tess = []

for index, value in (enumerate(tics_short_periods)):
    try: 
        period_tess = eleanor_plots(value, index)
        ls_tess.append(period_tess[0])
        pdm_tess.append(period_tess[1])
        exo_tess.append(period_tess[2])
    except: 
        ls_tess.append('nan')
        pdm_tess.append('nan')
        exo_tess.append('nan')

No eleanor postcard has been made for your target (yet). Using TessCut instead.




100%|██████████| 1000/1000 [00:00<00:00, 1756.46it/s]


No eleanor postcard has been made for your target (yet). Using TessCut instead.


100%|██████████| 1000/1000 [00:00<00:00, 1875.74it/s]


fitting to a limited range
No eleanor postcard has been made for your target (yet). Using TessCut instead.


...

fitting to a limited range
No eleanor postcard has been made for your target (yet). Using TessCut instead.


100%|██████████| 1000/1000 [00:00<00:00, 2075.00it/s]


fitting to a limited range
No eleanor postcard has been made for your target (yet). Using TessCut instead.


100%|██████████| 1000/1000 [00:00<00:00, 2131.70it/s]


No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for your target (yet). Using TessCut instead.
No eleanor postcard has been made for yo

In [12]:
len(ls_tess)

61

In [11]:
smallest_diff_tess = []
method_tess = []

for each, mcq in enumerate(short_periods): 
    diff_ls = np.abs(ls_tess[each] - mcq)
    diff_pdm = np.abs(pdm_tess[each] - mcq)
    diff_exo = np.abs(exo_tess[each] - mcq)
    
    if diff_ls > diff_pdm < diff_exo: 
        smallest_diff_tess.append(pdm_tess[each])
        method_tess.append('PDM')
            
    if diff_ls > diff_exo < diff_pdm: 
        smallest_diff_tess.append(exo_tess[each])
        method_tess.append('exoplanet acf')
        
    if diff_pdm > diff_ls < diff_exo: 
        smallest_diff_tess.append(ls_tess[each])
        method_tess.append('lomb-scargle')  

TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [ ]:
x =  short_periods
y = smallest_diff_tess

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = smallest_diff_tess, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

tess_periods_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title=" Tess Periods, Best Method",
   x_axis_label='McQuillan Period', y_axis_label='Tess Period', plot_width=900)

tess_periods_plot.line(x, x, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=x", line_width = 2)
tess_periods_plot.circle('x', 'y', source = source, color="mediumvioletred", line_color=None)
tess_periods_plot.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
tess_periods_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'pink', line_dash="dotted")

tess_periods_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("KIC ID", "@tess")] 

show(tess_periods_plot)

with open('smallest_diff_tess.data', 'wb') as filehandle:
    pickle.dump(smallest_diff_tess, filehandle)

with open('method_tess.data', 'wb') as filehandle:
    pickle.dump(method_tess, filehandle)

with open('ls_tess.data', 'wb') as filehandle:
    pickle.dump(ls_tess, filehandle)

with open('pdm_tess.data', 'wb') as filehandle:
    pickle.dump(pdm_tess, filehandle)

with open('exo_tess.data', 'wb') as filehandle:
    pickle.dump(exo_tess, filehandle)

In [ ]:
with open('smallest_diff_tess.data', 'rb') as filehandle:
    smallest_diff_tess = pickle.load(filehandle) 

In [ ]:
with open('method_tess.data', 'rb') as filehandle:
    method_tess = pickle.load(filehandle) 

In [ ]:
with open('ls_tess.data', 'rb') as filehandle:
    ls_tess = pickle.load(filehandle) 

In [ ]:
with open('pdm_tess.data', 'rb') as filehandle:
    pdm_tess = pickle.load(filehandle) 

In [ ]:
with open('exo_tess.data', 'rb') as filehandle:
    exo_tess = pickle.load(filehandle) 

In [ ]:
print(ls_tess[1], pdm_tess[1], exo_tess[1], short_periods[1])

In [ ]:
tess_kep_diff = []
for indx, value in enumerate(smallest_diff):
    difference = value - smallest_diff_tess[indx]
    tess_kep_diff.append(difference)

In [ ]:
x =  short_periods
y = tess_kep_diff

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = tess_kep_diff, 
    tess = (cat['tic_id'])[0:61],
    kep = (cat['kic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

diff_per_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-10, 10], title=" Difference Between Periods",
   x_axis_label='McQuillan Period', y_axis_label="Difference", plot_width=900)

diff_per_plot.line(x, 0, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=0", line_width = 2)
diff_per_plot.circle('x', 'y', source = source, color="mediumvioletred", line_color=None)

diff_per_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("KIC ID", "@kep"), 
    ("TIC ID", "@tess")] 

show(diff_per_plot)

In [13]:
for v, i in enumerate(kics_short_periods[20:]):
    kep_ac(i, v)

100%|██████████| 1000/1000 [00:01<00:00, 604.62it/s]


fitting to a limited range


100%|██████████| 1000/1000 [00:01<00:00, 511.77it/s]


KeyboardInterrupt: 

In [14]:
for v, i in enumerate(kics_short_periods[38:]):
    kep_ac(i, v)

100%|██████████| 1000/1000 [00:01<00:00, 547.59it/s]
